In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import numpy as np
# np.save('/gdrive/My Drive/global_images_1024.npy', Global_images_array)
# np.save('/gdrive/My Drive/local_images_512.npy', Local_images_array)

Local_images_array = np.load('/gdrive/My Drive/local_images_512.npy')
# Global_images_array = np.load('/gdrive/My Drive/global_images_1024.npy')

print(Local_images_array.shape)
# print(Global_images_array.shape)

(3200, 512, 512, 3)


In [ ]:
import numpy as np

b_label = [0] * 800
n_label = [1] * 800
inv_label = [2] * 800
ins_label = [3] * 800
y = b_label + n_label + inv_label + ins_label

num_classes = 4


In [ ]:
from keras.utils import to_categorical
labels = to_categorical(y)

In [ ]:
labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np

# Load pre-trained ResNet50 model without top layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# Create new model with custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in base model for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

87910968/87910968 [==============================] - 6s 0us/step


In [ ]:
import numpy as np

def split_data(X, y, test_size=0.2, random_state=5432):
    # Reshape X and y so that every 8 samples are treated as one datum
    n_datapoints = X.shape[0] // 8
    print(n_datapoints)
    X = X[:n_datapoints*8].reshape(n_datapoints, 8, X.shape[1], X.shape[2], X.shape[3])
    # X = X[:n_datapoints*8].reshape(n_datapoints, 8, X.shape[1])
    y = y[:n_datapoints*8].reshape(n_datapoints, 8, y.shape[1])
    
    # Split the data into training and testing sets
    n_test = int(n_datapoints * test_size)
    np.random.seed(random_state)
    indices = np.random.permutation(n_datapoints)
    X_train = X[indices[n_test:], :]
    y_train = y[indices[n_test:], :, :]
    X_test = X[indices[:n_test], :]
    y_test = y[indices[:n_test], :, :]
    
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = split_data(Local_images_array, labels)
X_train = X_train.reshape((320*8,512, 512, 3))
y_train = y_train.reshape((320*8,4))
X_test = X_test.reshape((80*8,512, 512, 3))
y_test = y_test.reshape((80*8,4))

400


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
160/160 [==============================] - 40s 156ms/step - loss: 13.8967 - accuracy: 0.3430
Epoch 2/10
160/160 [==============================] - 26s 160ms/step - loss: 4.1685 - accuracy: 0.3809
Epoch 3/10
160/160 [==============================] - 26s 164ms/step - loss: 2.4481 - accuracy: 0.4293
Epoch 4/10
160/160 [==============================] - 27s 168ms/step - loss: 2.0509 - accuracy: 0.4309
Epoch 5/10
160/160 [==============================] - 27s 170ms/step - loss: 1.7014 - accuracy: 0.4551
Epoch 6/10
160/160 [==============================] - 28s 172ms/step - loss: 1.5769 - accuracy: 0.4477
Epoch 7/10
160/160 [==============================] - 27s 171ms/step - loss: 1.2938 - accuracy: 0.4738
Epoch 8/10
160/160 [==============================] - 27s 171ms/step - loss: 1.2080 - accuracy: 0.4797
Epoch 9/10
160/160 [==============================] - 27s 171ms/step - loss: 1.2721 - accuracy: 0.4504
Epoch 10/10
160/160 [==============================] - 27s 172ms/step - 

In [ ]:
loss, accuracy = model.evaluate(test_data, test_labels)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

20/20 [==============================] - 32s 728ms/step - loss: 0.1954 - accuracy: 0.9281
Test loss: 0.1953846514225006
Test accuracy: 0.9281250238418579


In [ ]:
from keras.models import Model

feature_model = Model(inputs=model.input, outputs=model.layers[-2].output)
features_train = feature_model.predict(X_train)

80/80 [==============================] - 39s 425ms/step


In [ ]:
features_test = feature_model.predict(X_test)

20/20 [==============================] - 8s 433ms/step


In [ ]:
model.save("/gdrive/My Drive/Colab Notebooks/ResNet50_Model_Local_1.h5")

In [ ]:
np.save('/gdrive/My Drive/Colab Notebooks/ResNet50_Model_Local_1_LocalPatchSupervised_FeatsTrain.npy', features_train)
np.save('/gdrive/My Drive/Colab Notebooks/ResNet50_Model_Local_1_LocalPatchSupervised_FeatsTest.npy', features_test)

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
# fit model no training data
Xgb_clf = XGBClassifier()
Xgb_clf.fit(features_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
XGB_y_preds = Xgb_clf.predict(features_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, XGB_y_preds))

              precision    recall  f1-score   support

           0       0.75      0.63      0.68       208
           1       0.69      0.56      0.62       144
           2       0.72      0.42      0.53       144
           3       0.70      0.71      0.71       144

   micro avg       0.72      0.59      0.65       640
   macro avg       0.72      0.58      0.64       640
weighted avg       0.72      0.59      0.64       640
 samples avg       0.57      0.59      0.57       640



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
